In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, pipeline
import transformers
import torch
transformers.set_seed(42)
import dotenv
import wandb
import os

dotenv.load_dotenv("./.env", override=True)
wandb.login(key=os.getenv('WANDB_API_KEY'))
os.environ['WANDB_NOTEBOOK_NAME'] = 'interferenceQLoRa.ipynb'

/home/felix/llm/ChatTable/new_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: felix-ml (fml-team). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/felix/.netrc


In [3]:
from peft import PeftModel

model_name = "LeoLM/leo-hessianai-7b-chat"
# model_name = model.split("/")[-1]
adapter_name = "./LeoLM/leo-hessianai-7b-chat-ams-finetuned/"
#adapter_name = f"{model_name}-ams-finetuned"

In [4]:
model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto",
        max_memory={i: '24000MB' for i in range(torch.cuda.device_count())},
        # slower?
        load_in_4bit=True,
        #load_in_8bit=True,
        quantization_config=BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_compute_dtype=torch.float16,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_quant_type="nf4",        
        )
    )
model = PeftModel.from_pretrained(model, adapter_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
#move model to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Loading checkpoint shards: 100%|██████████| 2/2 [00:10<00:00,  5.23s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(32128, 4096, padding_idx=0)
        (layers): ModuleList(
          (0-31): 32 x LlamaDecoderLayer(
            (self_attn): LlamaSdpaAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=64, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=64, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
              )
              (k_proj): lora.Linear4bit(
                (base_l

In [5]:
project = f"{model_name.split('/')[-1]}"
run = wandb.init(project=f"{project}-ams-finetuned-interference", job_type="interference")

In [6]:
from transformers import StoppingCriteria, StoppingCriteriaList
import time

stop_token_ids = tokenizer.convert_tokens_to_ids(["<|endoftext|>"])

# define custom stopping criteria object
class StopOnTokens(StoppingCriteria):
    def __call__(self, input_ids: torch.LongTensor, scores: torch.FloatTensor, **kwargs) -> bool:
        for stop_id in stop_token_ids:
            if input_ids[0][-1] == stop_id:
                return True
        return False

stopping_criteria = StoppingCriteriaList([StopOnTokens()])

#device = "cuda:0"
pipe = transformers.pipeline(
    model               = model, 
    tokenizer           = tokenizer,
    return_full_text    = True,  # langchain expects the full text
    task                = 'text-generation',
    #device=device,
    device_map          = "auto",
    # we pass model parameters here too
    stopping_criteria   = stopping_criteria,    # without this model will ramble
    temperature         = 0.15,                 # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    top_p               = 0.15,                 # select from top tokens whose probability add up to 15%
    top_k               = 0,                    # select from top 0 tokens (because zero, relies on top_p)
    max_new_tokens      = 768*4,                # max number of tokens to generate in the output
    repetition_penalty  = 1.1                   # without this output begins repeating
)

def gen_text(system_prompt:str = None ,input_text:str = "hello", table = None):
    if system_prompt is None:
        system_prompt = "Du bist hilfst bei der Berufsberatung."
    messages = [
                {
                    "role": "system",
                    "content":system_prompt
                },
                {
                    "role": "user", 
                    "content": input_text
                },
            ]
    prompt = pipe.tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True
    )
    # Start the timer
    start_time = time.time()

    outputs = pipe(
        prompt,
        max_new_tokens=1024,
        do_sample=True,
        temperature=0.2,
        top_k=50,
        top_p=0.95
    )
    
    
    # End the timer
    end_time = time.time()
    duration = end_time - start_time

    # Calculate token lengths
    input_token_length = len(pipe.tokenizer.encode(prompt))
    output_token_length = len(pipe.tokenizer.encode(outputs[0]["generated_text"]))

    # Create W&B Table   
    table.add_data(system_prompt, input_text, outputs[0]["generated_text"], input_token_length, output_token_length, duration)

    # Log the table
    wandb.log({"Text Generation": table})

    
    return outputs[0]["generated_text"]

The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'ElectraForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FuyuForCausalLM', 'GitForCausalLM', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCausalLM', 'GPTNeoXForCausalLM', 'GPTNeoXJapaneseForCausalLM', 'GPTJForCausalLM', 'LlamaForCausalLM', 'MarianForCausalLM', 'MBartForCausalLM', 'MegaForCausalLM', 'MegatronBertForCausalLM', 'MistralForCausalLM', 'MixtralForCausalLM', 'MptForCausalLM', 'MusicgenForCausalLM', 'MvpForCausalLM', 'OpenLlamaForCausalLM', 'OpenAIGPTLMHeadModel', 'OPTForCausalLM', 'PegasusForCausalLM', 'PersimmonF

In [7]:
# Initialize the wandb.table outside of the function
columns = ["System Prompt", "Input Text", "Generated Text", "Input Token Length", "Output Token Length", "Generation Time"]
table = wandb.Table(columns=columns)


In [8]:
table

In [14]:
system = "Du bist hilfst bei der Berufsberatung. Du versuchst zu verstehen, was der Nutzer möchte und gibst eine Empfehlung ab. Gehe auf Tätigkeitsmerkmale, Anforderungen, Ausbildung, Beschäftigungsmöglichkeiten, Gehalt und Berufsausichten ein."
#input = "Ich bin 15 Jahre alt, gut im Umgang mit Menschen und möchte gerne einen Beruf erlernen, in dem ich viel mit Menschen zu tun habe."
input = [
    "Ich bin 15 Jahre alt, lese gerne und spiele gut Fußball. Wieviel Geld kann ich als Fußballer verdienen?",
    "Ich bin 30 Jahre alt und mag Tier sehr gerne. Ich möchte gerne einen Beruf erlernen, in dem ich viel mit Tieren zu tun habe.",
    "Ich bin 19 und möchte etwas technisches machen. Ich bin sehr gut in Mathe und Physik.",
]
for i in input:
    r = gen_text(system_prompt=system, input_text=i, table=table)
    print(r)
# r = gen_text(system_prompt=system, input_text=input, table=table)
# r

Ich bin 15 Jahre alt, lese gerne und spiele gut Fußball. Wieviel Geld kann ich als Fußballer verdienen?
Ich bin 30 Jahre alt und mag Tier sehr gerne. Ich möchte gerne einen Beruf erlernen, in dem ich viel mit Tieren zu tun habe.
Ich bin 19 und möchte etwas technisches machen. Ich bin sehr gut in Mathe und Physik.


In [16]:
wandb.finish()